In [1]:
import os

In [2]:
%pwd

'c:\\Users\\sandr\\PROJECTS_CODE\\car_sales_data_science_workflow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\sandr\\PROJECTS_CODE\\car_sales_data_science_workflow'

In [11]:
import pandas as pd

In [7]:
df = pd.read_csv("artifacts/data_ingestion/all_data.csv")

In [8]:
df.head()

,Order,Sales_in_thousands,Vehicle_type,Order Date,dealership,color,Power_perf_factor,Fuel_efficiency,Fuel_capacity,Engine_size,Horsepower,Wheelbase,Width,Length,Price_in_lakhs
0,4100134,4.85,car,2023-11-26,"404, Boston, MA, 02215",#24cc59,156.64710,21.8,19,2.53,200,94.08,71.17,168.15,17.73L
1,4100136,15.65,car,2023-07-17,"91, San Francisco, CA, 94016",#5c22fa,95.54393,23.5,23,3.02,265,102.77,79.34,196.93,4.69L
2,4100144,4.33,passenger,2023-11-21,"10, Portland, OR, 97035",#87f1c7,30.94899,19.5,15,5.46,177,83.29,68.25,233.17,21.77L
3,4100146,8.98,car,2023-12-17,"57, San Francisco, CA, 94016",#aec562,141.70133,18.2,25,1.52,218,97.74,86.04,212.64,23.29L
4,4100147,11.96,passenger,2023-10-28,"221, San Francisco, CA, 94016",#f2c704,30.04240,21.7,15,1.54,167,98.21,63.99,262.81,18.21L


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Order               1000 non-null   int64  
 1   Sales_in_thousands  1000 non-null   float64
 2   Vehicle_type        1000 non-null   object 
 3   Order Date          1000 non-null   object 
 4   dealership          1000 non-null   object 
 5   color               1000 non-null   object 
 6   Power_perf_factor   1000 non-null   float64
 7   Fuel_efficiency     1000 non-null   float64
 8   Fuel_capacity       1000 non-null   int64  
 9   Engine_size         1000 non-null   float64
 10  Horsepower          1000 non-null   int64  
 11  Wheelbase           1000 non-null   float64
 12  Width               1000 non-null   float64
 13  Length              1000 non-null   float64
 14  Price_in_lakhs      1000 non-null   object 
dtypes: float64(7), int64(3), object(5)
memory usage: 117.3+ 

In [11]:
df.isnull().sum()

Order                 0
Sales_in_thousands    0
Vehicle_type          0
Order Date            0
dealership            0
color                 0
Power_perf_factor     0
Fuel_efficiency       0
Fuel_capacity         0
Engine_size           0
Horsepower            0
Wheelbase             0
Width                 0
Length                0
Price_in_lakhs        0
dtype: int64

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            all_schema=schema,
            unzip_data_dir=config.unzip_data_dir 
        )

        return data_validation_config

In [8]:
import os
from mlProject import logger

In [9]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

In [13]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2023-08-01 20:22:54,619: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-01 20:22:54,621: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-01 20:22:54,631: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-08-01 20:22:54,631: INFO: common: created directory at: artifacts]
[2023-08-01 20:22:54,635: INFO: common: created directory at: artifacts/data_validation]
